In [1]:
system("sudo apt-get -y install libmagick++-dev", intern=TRUE)
install.packages("magick", verbose=TRUE)
install.packages("RWeka")

In [2]:
# Libraries
library(readr)        # reads in CSV
library(ggplot2)      # plot library
library(tidyverse)    # for data manipulation
library(gridExtra)    # multiple plots in 1
library(magick)       # attach dope image for visual
library(scales)       # show the colors
library(ggrepel)      # for graph repel (labels)
library(repr)         # resize graphs
library(hexbin)       # for hive scatter
library(naniar)       # to check for missing data
library(lubridate)    # for date and time
library(tm)
library(wordcloud)    # beautiful wordclouds
library(wordcloud2)
library(tidytext)     # text preprocessing
library(textdata)     # text preprocessing
library(reshape2)
library(knitr)
library(grid)
library(ggraph)
library(ggsci)
library(devtools)
library(circlize)
library(radarchart)
library(stringr)
library(sjmisc)
library(magick)
library(htmlwidgets)
library(VIM)          # missing values visual
library(colorspace)   # maybe for wordcloud
library(RWeka)
library(textmineR)

In [3]:
options(repr.plot.width=15, repr.plot.height=7)

# Custom Color Palette
my_colors <- c("#05A4C0", "#85CEDA", "#D2A7D8", "#A67BC5", "#BB1C8B", "#8D266E")
show_col(my_colors, labels = F, borders = NA)


# Custom Theme Variable
my_theme <- theme(plot.background = element_rect(fill = "grey98", color = "grey20"),
                  panel.background = element_rect(fill = "grey98"),
                  panel.grid.major = element_line(colour = "grey87"),
                  text = element_text(color = "grey20"),
                  plot.title = element_text(size = 22),
                  plot.subtitle = element_text(size = 17),
                  axis.title = element_text(size = 15),
                  axis.text = element_text(size = 15),
                  legend.box.background = element_rect(color = "grey20", fill = "grey98", size = 0.1),
                  legend.box.margin = margin(t = 3, r = 3, b = 3, l = 3),
                  legend.title = element_blank(),
                  legend.text = element_text(size = 15),
                  strip.text = element_text(size=17))

#  Importing Lexicons for sentiment Analysis 
A lexicon is a dictionary of words that computes the sentiment of a words by analyzing the "semantic orientation" of that word in a text. These codings are made by people, through crowdsorcing, etc.

* Afinn: gives the words a number between [-5, 5], where -5 means that the words is very negative and 5 means that the words is very positive.
* Bing: gives the words an assignment of positive/negative sentiment
* NRC: assigns the words one of the 8 primary emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and 2 sentiments (positive and negative)

In [4]:
afinn <- read_csv("../input/bing-nrc-afinn-lexicons/Afinn.csv",
                  col_types = cols(word = col_character(), value = col_double()))
bing <- read_csv("../input/bing-nrc-afinn-lexicons/Bing.csv",
                 col_types = cols(word = col_character(), sentiment = col_character()))
nrc <- read_csv("../input/bing-nrc-afinn-lexicons/NRC.csv",
                col_types = cols(word = col_character(), sentiment = col_character()))

# OUR DATA

In [5]:
# Data Wrangling and Visualization
library(glue)
library(cowplot)
library(magrittr)
library(plotly)
library(tidyverse)
library(widyr)
# Date & Time Manipulation.
library(hms)
library(lubridate) 
# Text Mining
library(tidytext)
library(tm)
library(wordcloud)
# Network Analysis
library(igraph)
# Network Visualization (D3.js)
library(networkD3)
# Set notebook directory.
MAIN.DIR <- here::here()

In [6]:
data <- read_csv("../input/ourcoviddata/FInalCovidData.csv");
show_col_types = TRUE

In [7]:
spec(data)

In [8]:
aggr(data)

In [9]:
sum(duplicated(data$comments))

In [10]:
distinct(data,comments, .keep_all= TRUE)



In [11]:
data %>% 
  # We  do not want to display accounts.
  filter(!str_detect(string = comments, pattern = '@')) %>% 
  head()

# Let us see the structure of this tibble

In [12]:
data %>% glimpse()

Time series COVID Tweets Analysis

In [13]:
options(repr.plot.width=15, repr.plot.height=9)

data %>% 
    select(Date) %>% 
    mutate(date =as.Date(data$Date, "%m/%d/%Y")) %>% 
    group_by(date) %>% 
    summarize(n = n(), .groups = "drop_last") %>%

    ggplot(aes(x=date, y = n)) + 
    geom_line(size = 1.5, color = my_colors[1]) +
    coord_cartesian(clip = 'off') +
    my_theme + theme(axis.title.x = element_blank()) +
    labs(title = "Number of Tweets in Time", subtitle = "2021", y = "Frequency")

#  Text Normalization

* the punctuation - removePunctuation
* extra white space - stripWhitespace
* transforms to lower case - tolower
* stopwords (common words that should be ignored) - stopwords
* numbers - removeNumbers

In [14]:
clean.data <- data %>% 
  # Remove column.
  select(-  Date) %>% 
  # Convert to lowercase. 
  mutate(Text = comments %>% str_to_lower) %>% 
  # Remove unwanted characters. 
  mutate(Text= comments %>% str_remove_all(pattern = '\\n')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = '&amp')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = 'https://t.co/[a-z,A-Z,0-9]*')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = 'http://t.co/[a-z,A-Z,0-9]*')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = 'https')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = 'http')) %>% 
  # Remove hashtags.
  mutate(Text = comments %>% str_remove_all(pattern = '#[a-z,A-Z]*')) %>% 
  # Remove accounts.
  mutate(Text = comments %>% str_remove_all(pattern = '@[a-z,A-Z]*')) %>% 
  # Remove retweets.
  mutate(Text = comments %>% str_remove_all(pattern = 'rt [a-z,A-Z]*: ')) %>% 
  mutate(Text = comments %>% str_remove(pattern = '(rt)')) %>% 
  mutate(Text = comments %>% str_remove_all(pattern = '\\_')) 

# Replace accents. 
replacement.list <- list('á' = 'a', 'é' = 'e', 'í' = 'i', 'ó' = 'o', 'ú' = 'u')

clean.data$Text <-  gsub("https\\S*", "", clean.data$Text)
clean.data$Text <-  gsub("@\\S*", "", clean.data$Text) 
clean.data$Text  <-  gsub("amp", "", clean.data$Text) 
clean.data$Text  <-  gsub("[\r\n]", "", clean.data$Text)
clean.data$Text  <-  gsub("[[:punct:]]", "", clean.data$Text)
clean.data$Text  <- gsub("RT", "", clean.data$Text)
clean.data$Text <- gsub('[0-9]+', '',clean.data$Text)

clean.data %<>% 
  mutate(Text = chartr(old = names(replacement.list) %>% str_c(collapse = ''), 
                       new = replacement.list %>% str_c(collapse = ''),
                       x = Text))


clean.data %>% head(5)


In [15]:
corpus <-  Corpus(x = VectorSource(x = clean.data$Text))

In [16]:
data.text <- corpus %>% 
  tm_map(removePunctuation) %>% 
  tm_map(removeNumbers) %>% 
  tm_map(removeWords, stopwords('english')) %>% 
  tm_map(PlainTextDocument) # %>% 
  # We could also use stemming by uncommenting the folowing line. 
  # tm_map(stemDocument, 'english')

# Recover data into original tibble.
clean.data %<>% mutate(Text = data.text[[1]]$content)

# Get Hashtags

In [17]:
GetHashtags <- function(tweet) {

  hashtag.vector <- str_extract_all(string = tweet, pattern = '#\\S+', simplify = TRUE) %>% 
    as.character()
  
  hashtag.string <- NA
  
  if (length(hashtag.vector) > 0) {
    
    hashtag.string <- hashtag.vector %>% str_c(collapse = ', ')
    
  } 

  return(hashtag.string)
}


In [18]:
hashtags.df <- tibble(
  Hashtags = data$comments %>% map_chr(.f = ~ GetHashtags(tweet = .x))
)

In [19]:
hashtags.df %>% head()

In [20]:
clean.data %<>% bind_cols(hashtags.df)

# Word Cloud

In [21]:
# Remove the shortcut 'q' for 'que'.
extra.stop.words <- c('q')

stopwords.df <- tibble(
  word = c(stopwords(kind = 'en'),  
           extra.stop.words)
  )

words.df <- clean.data %>% 
  unnest_tokens(input = Text, output = word) %>%  #split the sentences into words/token
  anti_join(y = stopwords.df, by = 'word')

word.count <- words.df %>% count(word, sort = TRUE)

word.count %>% head(10)

In [22]:
plt <- word.count %>% 
  # Set count threshold. 
  filter(n > 600) %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(x = word, y = n)) +
  theme_light() + 
  geom_col(fill = 'pink', alpha = 0.8) +
  xlab(NULL) +
  coord_flip() +
  ggtitle(label = 'Top Word Count')

plt %>% ggplotly()

In [23]:
wordcloud(
  words = word.count$word, 
  freq = word.count$n, 
  min.freq = 100,max.words=500, random.order=FALSE, rot.per=0.35,scale=c(5,1),
  colors = brewer.pal(8, 'Dark2')
)



# Hashtags

In [24]:
hashtags.unnested.df <- clean.data %>% 
  select(Hashtags) %>% 
  unnest_tokens(input = Hashtags, output = hashtag)
  
hashtags.unnested.count <- hashtags.unnested.df %>% 
  count(hashtag) %>% 
  drop_na()

In [25]:
wordcloud(
  words = str_c('#',hashtags.unnested.count$hashtag), 
  freq = hashtags.unnested.count$n, 
  min.freq = 20, max.words=500, random.order=FALSE, rot.per=0.35,scale=c(5,1),
  colors=brewer.pal(8, 'Dark2')
)

# Sentiment Analysis

In [26]:
options(repr.plot.width=15, repr.plot.height=15)
unnest_sentiments <- data %>% 
    mutate(text = as.character(data$comments)) %>% 
    unnest_tokens(word, text)


unnest_sentiments %>% 
    inner_join(bing, by="word") %>%
    count(word, sentiment, sort=T) %>% 
    acast(word ~ sentiment, value.var = "n", fill=0) %>% 
  
    # wordcloud
    comparison.cloud(colors=my_colors[c(5, 1)], max.words = 400, title.size = 2,
                  scale = c(3,0.5))

In [27]:
options(repr.plot.width=15, repr.plot.height=9)

# The plot:
unnest_sentiments %>% 
    inner_join(nrc, "word") %>%
    filter(!sentiment %in% c("positive", "negative")) %>% 
    count(sentiment, sort=T) %>% 

    ggplot(aes(x=reorder(sentiment, n), y=n)) +
    geom_bar(stat="identity", aes(fill=n), show.legend=F) +
    geom_label(aes(label=format(n, big.mark = ",")), size=5, fill="white") +
    labs(x="Sentiment", y="Frequency", title="What is the overall mood in Tweets?") +
    scale_fill_gradient(low = my_colors[3], high = my_colors[1], guide="none") +
    coord_flip() + 
    my_theme + theme(axis.text.x = element_blank())

1. Trust: good, school, safe, hospital
1. Fear: pandemic, death, government, sick
1. Anticipation: time, daily, watch

In [28]:
unnest_sentiments %>% 
  inner_join(nrc, "word") %>% 
  count(sentiment, word, sort=T) %>%
  group_by(sentiment) %>% 
  arrange(desc(n)) %>% 
  slice(1:7) %>% 
  
  # Plot:
  ggplot(aes(x=reorder(word, n), y=n)) +
  geom_col(aes(fill=sentiment), show.legend = F) +
  facet_wrap(~sentiment, scales = "free_y", nrow = 2, ncol = 5) +
  coord_flip() +
  my_theme + theme(axis.text.x = element_blank()) +
  labs(x="Word", y="Frequency", title="Sentiment split by most frequent words") +
  scale_fill_manual(values = c(my_colors, "#BE82AF", "#9D4387", "#DEC0D7",
                                 "#40BDC8", "#80D3DB", "#BFE9ED"))

In [29]:
options(repr.plot.width=15, repr.plot.height=9)

unnest_sentiments %>% 
  # by word and value count number of occurences
  inner_join(afinn, "word") %>% 
  count(word, value, sort=T) %>% 
  mutate(contribution = n * value,
         sentiment = ifelse(contribution<=0, "Negative", "Positive")) %>% #another variable
  arrange(desc(abs(contribution))) %>% 
  head(20)  %>% 
  
  # plot
  ggplot(aes(x=reorder(word, contribution), y=contribution, fill=sentiment)) +
  geom_col(aes(fill=sentiment), show.legend = F) +
  labs(x="Word", y="Contribution", title="Words with biggest contributions in positive/negative sentiments") +
  coord_flip() +
  scale_fill_manual(values=my_colors[c(3, 2)]) + 
  my_theme

# Network Analysis

In [30]:
bi.gram.words <- clean.data %>% 
  unnest_tokens(
    input = Text, 
    output = bigram, 
    token = 'ngrams', 
    n = 2
  ) %>% 
  filter(! is.na(bigram))

bi.gram.words %>% 
  select(bigram) %>% 
  head(10)

In [31]:
bi.gram.words %<>% 
  separate(col = bigram, into = c('word1', 'word2'), sep = ' ') %>% 
  filter(! word1 %in% stopwords.df$word) %>% 
  filter(! word2 %in% stopwords.df$word) %>% 
  filter(! is.na(word1)) %>% 
  filter(! is.na(word2)) 

In [32]:
bi.gram.count <- bi.gram.words %>% 
  count(word1, word2, sort = TRUE) %>% 
  # We rename the weight column so that the 
  # associated network gets the weights (see below).
  rename(weight = n)

bi.gram.count %>% head()

In [33]:
# Distinct Item1
sources <- bi.gram.count %>% 
                distinct(word1) %>% 
                rename(label = word1)

# Distinct item2
destinations <- bi.gram.count %>% 
                    distinct(word2) %>% 
                    rename(label = word2)


* nodes are the unique words - each word has an identification ID
* edges are the bigrams, meaning that they show how frequently we find a combination of 2 words (represented by their unique ID)

In [34]:
# ----- NODES -----
# Unique Items + create unique ID
nodes <- full_join(sources, destinations, by="label") %>% rowid_to_column("id")
nodes



In [35]:
# ----- EDGES -----
# Adds unique ID of Item 1 to data
edges <- bi.gram.count %>% 
            left_join(nodes, by = c("word1" = "label")) %>% 
            rename(from = id)

# Adds unique ID of Item 2 to data
edges <- edges %>% 
            left_join(nodes, by = c("word2" = "label")) %>% 
            rename(to = id) %>% 
            rename(weight = weight)

# Select only From | To | Weight (frequency)
edges <- edges %>% select(from, to, weight)

In [36]:
nodes %>% head(3)
edges %>% head(3)

In [37]:
# Export the nodes & edges data - 
write.csv(nodes,"nodes.csv", row.names = FALSE)
write.csv(edges,"edges.csv", row.names = FALSE)

In [38]:
Rnetwork <- graph_from_data_frame(d=edges, vertices=nodes, directed=T)
deg <- degree(
  Rnetwork,
  v = V(Rnetwork),
  mode = c("all"),
  loops = TRUE,
  normalized = FALSE
)
deg
degree_distribution(Rnetwork)
plot(degree_distribution(Rnetwork))

In [39]:
hist(deg, breaks=1:vcount(Rnetwork)-1, main="Histogram of node degree")
ggplot(mapping = aes(x = deg)) +
    theme_light() +

geom_histogram(fill = 'blue', bins = 30)

In [40]:

cat("Total Edges count :")
ecount(Rnetwork)
cat("Total Nodes count :")
vcount(Rnetwork)
cat("Average path Length of a Graph :")
average.path.length(Rnetwork)
cat("Maximum Degree of a Node in a Network :")
max(degree(Rnetwork))
cat("Minimum Degree of a Node in a Network :")
min(degree(Rnetwork))
cat("Eccentricity of a Graph :")
eccentricity(Rnetwork,vids = V(Rnetwork),mode = c("all","Out","In","Total"))


In [41]:

Rnetwork.ego.networks = make_ego_graph(graph = Rnetwork, nodes = c(9))
ego9 = Rnetwork.ego.networks[[1]]

# plot(ego9)
# wc <- cluster_walktrap(ego9)
# members <- membership(wc)
# karate_d3 <- igraph_to_networkD3(ego9, group = members)
#  #Create force directed network plot
# forceNetwork(Links = karate_d3$links, Nodes = karate_d3$nodes, 
#              Source = 'source', Target = 'target', 
#              NodeID = 'name', Group = 'group',
#             fontSize = 12,
#   zoom = TRUE, 
#   opacityNoHover = 1  )
selegoV <- ego(Rnetwork, order=1, nodes = 9, mode = "all", mindist = 0)
# turn the returned list of igraph.vs objects into a graph
selegoG <- induced_subgraph(Rnetwork,unlist(selegoV))

# plot the subgraph
plot(selegoG,vertex.label=V(selegoG)$name)

In [42]:

# Compute the centrality measures for the biggest connected component from above.
node.impo.df <- tibble(
  word = V(Rnetwork)$label,  
  degree = strength(graph = Rnetwork),
  closeness = closeness(graph = Rnetwork), 
  betweenness = betweenness(graph = Rnetwork),

    
)
# Degree centrality
node.impo.df %>% 
  arrange(- degree) %>%
  head(10)
cat("Eigen Centrality ")
eigen_centrality(Rnetwork)

In [43]:
threshold <- 150

# For visualization purposes we scale by a global factor. 
ScaleWeight <- function(x, lambda) {
  x / lambda
}

network <-  bi.gram.count %>%
  filter(weight > threshold) %>%
  mutate(weight = ScaleWeight(x = weight, lambda = 2E3)) %>% 
  graph_from_data_frame(directed=T) 

Rnetwork <- graph_from_data_frame(d=edges, vertices=nodes, directed=T) 

In [44]:
Rnetwork

In [45]:
is.weighted(network)

In [46]:
# Store the degree.
V(network)$degree <- strength(graph = network)

# Compute the weight shares.
E(network)$width <- E(network)$weight/max(E(network)$weight)

plot(
  network, 
  vertex.color = 'lightblue',
  # Scale node size by degree.
  vertex.size = 20*V(network)$degree,
  vertex.label.color = 'black', 
  vertex.label.cex = 1, 
  vertex.label.dist = 1.6,
  edge.color = 'gray', 
  # Set edge width proportional to the weight relative value.
  edge.width = 4*E(network)$width ,
  main = 'Bigram Count Network', 
  sub = glue('Weight Threshold: {threshold}'), 
  alpha = 50
)

In [47]:
# Get all connected components.
clusters(graph = network)

In [48]:
# Select biggest connected component.  
V(network)$cluster <- clusters(graph = network)$membership

cc.network <- induced_subgraph(
  graph = network,
  vids = which(V(network)$cluster == which.max(clusters(graph = network)$csize))
)

cc.network 

In [49]:
# Store the degree.
V(cc.network)$degree <- strength(graph = cc.network)

# Compute the weight shares.
E(cc.network)$width <- E(cc.network)$weight/max(E(cc.network)$weight)

 plot(
  cc.network, 
  vertex.color = 'lightblue',
  # Scale node size by degree.
  vertex.size = 10*V(cc.network)$degree,
  vertex.label.color = 'black', 
  vertex.label.cex = 1, 
  vertex.label.dist = 1.6,
  edge.color = 'gray', 
  # Set edge width proportional to the weight relative value.
  edge.width = 3*E(cc.network)$width ,
  main = 'Bigram Count Network (Biggest Connected Component)', 
  sub = glue('Weiight Threshold: {threshold}'), 
  alpha = 50
)

In [50]:
# Treshold
threshold <- 10

network <-  bi.gram.count %>%
  filter(weight > threshold) %>%
  graph_from_data_frame(directed = FALSE)
# network <- graph_from_data_frame(d=edges, vertices=nodes, directed=T)
# Store the degree.
V(network)$degree <- strength(graph = network)
# Compute the weight shares.
E(network)$width <- E(network)$weight/max(E(network)$weight)

# Create networkD3 object.
network.D3 <- igraph_to_networkD3(g = network)
# Define node size.
network.D3$nodes %<>% mutate(Degree = (1E-2)*V(network)$degree)
# Degine color group (I will explore this feature later).
network.D3$nodes %<>% mutate(Group = 1)
# Define edges width. 
network.D3$links$Width <- 10*E(network)$width

forceNetwork(
  Links = network.D3$links, 
  Nodes = network.D3$nodes, 
  Source = 'source', 
  Target = 'target',
  NodeID = 'name',
  Group = 'Group', 
  opacity = 0.9,
  Value = 'Width',
  Nodesize = 'Degree', 
  # We input a JavaScript function.
  linkWidth = JS("function(d) { return Math.sqrt(d.value); }"), 
  fontSize = 12,
  zoom = TRUE, 
  opacityNoHover = 1
)

#  Community Detection

In [51]:
threshold <- 80

# For visualization purposes we scale by a global factor. 
ScaleWeight <- function(x, lambda) {
  x / lambda
}

Rnetwork <-  bi.gram.count %>%
  filter(weight > threshold) %>%
  mutate(weight = ScaleWeight(x = weight, lambda = 2E3)) %>% 
  graph_from_data_frame(directed=T) 

# Community Building using Edge Betweeness

In [52]:
ceb <- cluster_edge_betweenness(network)
dendPlot(ceb, mode="hclust")


In [53]:
plot(ceb, network)


In [54]:
class(ceb)
membership(ceb)

In [55]:
# cfg <- cluster_fast_greedy(as.undirected(network))
# plot(cfg, as.undirected(network))

In [56]:
# V(network)$community <- cfg$membership
# colrs <- adjustcolor( c("gray50", "tomato", "gold", "yellowgreen"), alpha=.6)
# plot(network, vertex.color=colrs[V(network)$community])


# Community Detection Using Skip Gram

In [57]:
skip.window <- 2

skip.gram.words <- clean.data %>% 
  unnest_tokens(
    input = Text, 
    output = skipgram, 
    token = 'skip_ngrams', 
    n = skip.window
  ) %>% 
  filter(! is.na(skipgram))

In [58]:
 clean.data %>% 
  slice(4) %>% 
  pull(Text)

In [59]:
skip.gram.words %>% 
  select(skipgram) %>% 
  slice(10:20)

In [60]:
skip.gram.words$num_words <- skip.gram.words$skipgram %>% 
  map_int(.f = ~ ngram::wordcount(.x))

skip.gram.words %<>% filter(num_words == 2) %>% select(- num_words)

skip.gram.words %<>% 
  separate(col = skipgram, into = c('word1', 'word2'), sep = ' ') %>% 
  filter(! word1 %in% stopwords.df$word) %>% 
  filter(! word2 %in% stopwords.df$word) %>% 
  filter(! is.na(word1)) %>% 
  filter(! is.na(word2)) 

skip.gram.count <- skip.gram.words  %>% 
  count(word1, word2, sort = TRUE) %>% 
  rename(weight = n)

skip.gram.count %>% head()

In [61]:
# Treshold
threshold <- 20

network <-  skip.gram.count %>%
  filter(weight > threshold) %>%
  graph_from_data_frame(directed = FALSE)

# Select biggest connected component.  
V(network)$cluster <- clusters(graph = network)$membership

cc.network <- induced_subgraph(
  graph = network,
  vids = which(V(network)$cluster == which.max(clusters(graph = network)$csize))
)

# Store the degree.
V(cc.network)$degree <- strength(graph = cc.network)
# Compute the weight shares.
E(cc.network)$width <- E(cc.network)$weight/max(E(cc.network)$weight)

# Create networkD3 object.
network.D3 <- igraph_to_networkD3(g = cc.network)
# Define node size.
network.D3$nodes %<>% mutate(Degree = (1E-2)*V(cc.network)$degree)
# Degine color group (I will explore this feature later).
network.D3$nodes %<>% mutate(Group = 1)
# Define edges width. 
network.D3$links$Width <- 10*E(cc.network)$width

forceNetwork(
  Links = network.D3$links, 
  Nodes = network.D3$nodes, 
  Source = 'source', 
  Target = 'target',
  NodeID = 'name',
  Group = 'Group', 
  opacity = 0.9,
  Value = 'Width',
  Nodesize = 'Degree', 
  # We input a JavaScript function.
  linkWidth = JS("function(d) { return Math.sqrt(d.value); }"), 
  fontSize = 12,
  zoom = TRUE, 
  opacityNoHover = 1
)

In [62]:
comm.det.obj <- cluster_louvain(
  graph = network, 
  weights = E(network)$weight
)

In [63]:
comm.det.obj

In [64]:
V(cc.network)$membership <- membership(comm.det.obj)

In [65]:
# We use the membership label to color the nodes.
network.D3$nodes$Group <- V(cc.network)$membership

forceNetwork(
  Links = network.D3$links, 
  Nodes = network.D3$nodes, 
  Source = 'source', 
  Target = 'target',
  NodeID = 'name',
  Group = 'Group', 
  opacity = 0.9,
  Value = 'Width',
  Nodesize = 'Degree', 
  # We input a JavaScript function.
  linkWidth = JS("function(d) { return Math.sqrt(d.value); }"), 
  fontSize = 12,
  zoom = TRUE, 
  opacityNoHover = 1
)